In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [2]:
df = pd.read_csv('data.csv' , encoding='latin-1')

In [3]:
print("Dataset Shape:", df.shape)

Dataset Shape: (541909, 8)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [7]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [10]:
# Separate cancelled and regular transactions  
cancelled_df = df[df['Quantity'] < 0].copy()
regular_df = df[df['Quantity'] > 0].copy()

# For cancelled items, create match key using ABSOLUTE value of quantity
cancelled_df['match_key'] = (
    cancelled_df['CustomerID'].astype(str) + '_' + 
    cancelled_df['StockCode'].astype(str) + '_' + 
    cancelled_df['Quantity'].abs().astype(str) + '_' +  # Use abs() here
    cancelled_df['UnitPrice'].astype(str)
)

# For regular items, use actual quantity (already positive)
regular_df['match_key'] = (
    regular_df['CustomerID'].astype(str) + '_' + 
    regular_df['StockCode'].astype(str) + '_' + 
    regular_df['Quantity'].astype(str) + '_' +          # Don't use abs() here
    regular_df['UnitPrice'].astype(str)
)

# Remove regular transactions that have corresponding cancellations
cancelled_keys = set(cancelled_df['match_key'])
df = regular_df[~regular_df['match_key'].isin(cancelled_keys)].copy()
df = df.drop('match_key', axis=1)

print(f"Removed {len(regular_df) - len(df)} cancelled transactions")

# Test with your specific example
test_key = "16446_23843_80995_2.08"
print(f"Cancelled keys contain test case: {test_key in cancelled_keys}")

# Verify the specific transaction is gone
check = df[
    (df['CustomerID'] == 16446) & 
    (df['StockCode'] == '23843') & 
    (df['Quantity'] == 80995)
]
print(f"Original transaction still exists: {len(check) > 0}")

Removed 0 cancelled transactions
Cancelled keys contain test case: False
Original transaction still exists: False


In [11]:
df = df.dropna(subset='CustomerID')

In [12]:
df.shape

(392695, 8)

In [13]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
duplicate_line_items = df.duplicated(subset=['InvoiceNo', 'StockCode']).sum()
print(f"Duplicate line items (same product in same invoice): {duplicate_line_items}")

Duplicate line items (same product in same invoice): 9871


In [15]:
if duplicate_line_items > 0:
    duplicates = df[df.duplicated(subset=['InvoiceNo', 'StockCode'], keep=False)]
    print(duplicates[['InvoiceNo', 'StockCode', 'Description', 'Quantity']].head())

    InvoiceNo StockCode                      Description  Quantity
113    536381     71270                  PHOTO CLIP LINE         1
125    536381     71270                  PHOTO CLIP LINE         3
483    536409    90199C  5 STRAND GLASS NECKLACE CRYSTAL         3
485    536409     22111     SCOTTIE DOG HOT WATER BOTTLE         1
489    536409     22866    HAND WARMER SCOTTY DOG DESIGN         1


In [16]:
df = df.drop_duplicates(subset=['InvoiceNo', 'StockCode'], keep='first')
print(f"Removed {duplicate_line_items} duplicate line items")

Removed 9871 duplicate line items


In [17]:
df = df[df["Quantity"] > 0]

In [18]:
df = df[df["UnitPrice"] > 0]

In [19]:
print("Cleaned dataset shape:", df.shape)

Cleaned dataset shape: (382788, 8)


In [20]:
df.to_csv('cleaned_data.csv', index=False)